<a href="https://colab.research.google.com/github/Chuck2Win/NH_project/blob/main/%5BTEST%5D.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
! pip3 install transformers
! pip3 install imbalanced-learn
! pip3 install kobert-transformers
! pip3 install sentencepiece

     |████████████████████████████████| 1.8MB 13.1MB/s 
     |████████████████████████████████| 2.9MB 63.3MB/s 
     |████████████████████████████████| 890kB 52.9MB/s 
  Created wheel for sacremoses: filename=sacremoses-0.0.43-cp36-none-any.whl size=893261 sha256=4a9627444457d8dabf63c113c5688aba3605e967b85a8d549e260d45352147a7
  Stored in directory: /root/.cache/pip/wheels/29/3c/fd/7ce5c3f0666dab31a50123635e6fb5e19ceb42ce38d4e58f45
Successfully built sacremoses
     |████████████████████████████████| 1.2MB 13.1MB/s 


In [2]:
import pandas as pd
import numpy as np
import re
import torch
import torch.nn as nn
import torch.nn.functional as F
import os
import torchtext
from google.colab import drive
from transformers import AdamW,get_linear_schedule_with_warmup,get_constant_schedule_with_warmup
import time
import datetime
from torch.utils.data import TensorDataset, DataLoader
from pandas import DataFrame as df
import time,datetime
from sklearn.metrics import classification_report
from tqdm import tqdm
drive.mount('/content/gdrive')
os.chdir('./gdrive/My Drive/con')
import copy
from kobert_transformers import get_kobert_model,get_tokenizer

Mounted at /content/gdrive


# 불러오기

In [3]:
test_data = pd.read_csv('./news_test (1).csv')

## 시간 측정 시작

In [4]:
import time 
now = time.time()

# 모델

In [5]:
class kobert_classifier(nn.Module):
    def __init__(self, kobert):
        super().__init__()
        self.bert = kobert
        self.classifier = nn.Linear(768+3,2)
    def forward(self,input_ids,attention_mask,length, longer, shorter):
        output = self.bert.forward(input_ids = input_ids, attention_mask = attention_mask)
        input = torch.cat([output.pooler_output, length.unsqueeze(-1),longer.unsqueeze(-1) ,shorter.unsqueeze(-1)],-1) # bs, 768
        predict = self.classifier.forward(input)
        return predict

# 전처리

In [6]:
tokenizer = get_tokenizer()
kobert = get_kobert_model()

In [7]:
def remove_stopword(data,stopword_list):
    pattern =''
    for i in stopword_list:
        pattern+='%s|'%i
    return re.sub(pattern[:-1],'',data)
# 대괄호 안의 문자 제거
# stop word 제거
stopword_list = ['재배포 금지','무단배포', '무단전재']
# 대괄호와 그 안의 내용 제거
test_data['content'] = test_data['content'].apply(lambda i : re.sub(r'\[[^)]*\]', '', i)) 
# 소괄호와 그 안의 내용 제거
test_data['content'] = test_data['content'].apply(lambda i : re.sub(r'\([^)]*\)', '', i)) 
# stop word 제거
test_data['content'] = test_data['content'].apply(lambda i : remove_stopword(i,stopword_list))
# 길이
test_data['length'] = test_data['content'].apply(lambda i : len(tokenizer.encode(i)))

Token indices sequence length is longer than the specified maximum sequence length for this model (615 > 512). Running this sequence through the model will result in indexing errors


## BERT에 맞게끔 변경

In [8]:
# max len으로 자르고, 모자란 부분은 패딩으로 채움
test_data['ids'] = test_data['content'].apply(lambda i : tokenizer.encode(i,add_special_tokens=True,truncation=True,padding='max_length',max_length=64))
# attention mask - mask될 부분은 0, 아닌 부분은 1
test_data['mask']=(torch.tensor(test_data['ids'].tolist()).eq(1)==0).long().tolist()
# longer
test_data['longer'] = test_data['length']>512
# shorter
test_data['shorter'] = test_data['length']<=512
test_data['longer'] = test_data['longer'].astype(int)
test_data['shorter'] = test_data['shorter'].astype(int)

In [9]:
batch_size = 128
test_data = TensorDataset(torch.LongTensor(test_data['ids'].tolist()), torch.LongTensor(test_data['mask'].tolist()), torch.LongTensor(test_data['length'].tolist()), torch.LongTensor(test_data['longer'].tolist()), torch.LongTensor(test_data['shorter'].tolist()))
test_dataloader = DataLoader(test_data, batch_size=batch_size,drop_last=False)

In [10]:
# 전처리까지의 걸린 시간
print(time.time()-now)

52.553717851638794


# Model

In [11]:
now = time.time()

In [12]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'
model = kobert_classifier(kobert).to(device)
model.load_state_dict(torch.load('./epoch_45_model')) # model load

<All keys matched successfully>

In [13]:
print(time.time()-now)

12.792293787002563


In [14]:
with torch.no_grad():
    model.eval()
    Predicted=[]
    for batch in test_dataloader:
        batch = tuple(t.to(device) for t in batch)
        input_ids, attention_mask, length, longer, shorter = batch
        outputs = model.forward(input_ids, attention_mask, length, longer, shorter)
        predicted = outputs.argmax(-1).tolist()
        Predicted.extend(predicted)
print(time.time()-now)

156.3186058998108
